This script computes the SSIM score for different head rotations. 
You need to install skimage, pandas, PIL, cv2 libreries

In [1]:
import os
from skimage.metrics import structural_similarity as ssim
import glob
import pandas as pd
from skimage import data, img_as_float
from PIL import Image
import os
# from deepface import DeepFace
import cv2
from pathlib import Path

Change following paths to match your ground-truth path (ground_truth_parent_directory) and path where the generated frames by your algorithm  are located (generated_parent_directory)

In [2]:
ground_truth_parent_directory = '3D_real_head_dataset/images/ground_truth/'
generated_parent_directory = 'DAGAN_results_3D_heads/'
#generated_parent_directory = 'FOM_results_3D_head/'


In [3]:

#Directories containing ground-truth 
ground_truth_directories = [ground_truth_parent_directory+d for d in os.listdir(ground_truth_parent_directory) 
                            if os.path.isdir(os.path.join(ground_truth_parent_directory, d))]
#Directories containing generated reenactment frames 
generated_directories = [generated_parent_directory+d for d in os.listdir(generated_parent_directory) 
                         if os.path.isdir(os.path.join(generated_parent_directory, d))] 

#For each ground-truth find all deepfake folders that contain the same head rotation.
#ground-truth folder is key and the value is gnerated folders (both contain the same head rotation)
gt_gen_path_dict = {}
for dir_gt in ground_truth_directories:
    rotation_type_gt = dir_gt.split('/')[-1].split('_')[0] #head rotation of ground-truth
    if rotation_type_gt not in gt_gen_path_dict:
                gt_gen_path_dict[dir_gt] = []
    #if the same head rotation is found in generated folder, add it as a value to dict
    for dir_gen in generated_directories: 
        rotation_type_gen = dir_gen.split('/')[-1].split('_')[0]
        
        if rotation_type_gen == rotation_type_gt:
            gt_gen_path_dict[dir_gt].append(dir_gen)

In [4]:
def compute_SSIM(orig_frame, fake_frame):
    """
    Compute SSIM score between video frames.
    Args:
    orig_frame: ground-truth frame.
    fake_frame: fake frame
    Returns:
    SSIM score between ground-truth and fake frame
    """
    ground_truth_image = img_as_float(Image.open(fake_frame))
    generated = img_as_float(Image.open(orig_frame))
  
    ssim_score = ssim(ground_truth_image,generated, data_range=ground_truth_image.max() -
                      ground_truth_image.min(), multichannel=True)  
    return ssim_score


def compute_ssim_per_frame(original_path, path1, path2, path3, path4):
    """
    Compute SSIM between ground truth and generated frames for one video (100 frames). We use 4 different 
    identities to generate the same head rotation 
    Args:
    original_path: path to ground-truth directory
    path1, path2, path3, path4: path to deepfake generated directories. Inside each path we have fake frames generated
    by different identities
    Returns:
    A list of SSIM scores between the ground-truth frames and fake frames. The score for each frame is the average
    over the four generated images.
    """
    ssim_list = []
    ground_truth_frames_212 = [original_path + '/' + d for d in sorted(os.listdir(original_path))]
    generated_frames_122 = [path1 + '/' + d for d in sorted(os.listdir(path1))]
    generated_frames_340 = [path2 + '/' + d for d in sorted(os.listdir(path2))]
    generated_frames_344 = [path3 + '/' + d for d in sorted(os.listdir(path3))]
    generated_frames_359 = [path4 + '/' + d for d in sorted(os.listdir(path4))]
    print(original_path)
    for ground_truth, gen_122, gen_340, gen_344, gen_359 in zip(ground_truth_frames_212, 
                                                            generated_frames_122, 
                                                            generated_frames_340,
                                                            generated_frames_344,
                                                            generated_frames_359):
        
        SSIM_score_122 = compute_SSIM(ground_truth, gen_122 )
        SSIM_score_340 = compute_SSIM(ground_truth, gen_340)
        SSIM_score_344 = compute_SSIM(ground_truth, gen_344)
        SSIM_score_359= compute_SSIM(ground_truth, gen_359)
        
        mean_ssim = (SSIM_score_122+SSIM_score_340+SSIM_score_344+SSIM_score_359)/4 
        ssim_list.append(mean_ssim)
    return ssim_list

In [5]:
def main():
    
    # Create a dataframe to store image name and SSIM score for different head rotation per frame
    dataframe = pd.DataFrame({'image_name': []})
    for groundtruth_path, generate_path  in gt_gen_path_dict.items():
        names = [d for d in sorted(os.listdir(groundtruth_path))] #video frame names
        dataframe["image_name"] = names 
        ssim_list = []
        rotatio_type = groundtruth_path.split('/')[-1].split('_')[0]
        print(rotatio_type)
        #In the paper we only evaluate the three head rotations below
        if rotatio_type == "pitchNegative": #head rotation of ground-truth
            ssim_list = compute_ssim_per_frame(groundtruth_path, generate_path[0], generate_path[1], 
                                                generate_path[2], generate_path[3])
            dataframe["pitchNegative"] = ssim_list
            
            
        if rotatio_type == "yawPositive": #head rotation of ground-truth
            ssim_list = compute_ssim_per_frame(groundtruth_path, generate_path[0], generate_path[1], 
                                                generate_path[2], generate_path[3])
            dataframe["yawPositive"] = ssim_list
            
                    
        if rotatio_type == "pitchYawPositive": #head rotation of ground-truth
            ssim_list = compute_ssim_per_frame(groundtruth_path, generate_path[0], generate_path[1], 
                                                generate_path[2], generate_path[3])
            dataframe["pitchYawPositive"] = ssim_list
    
    print(dataframe)   
    csv_path =   "cv_scores/"+generated_parent_directory
    if not os.path.exists(csv_path):
        os.makedirs(csv_path, exist_ok=True)
                
    dataframe.to_csv(csv_path+"ssim_scores_per_frame.csv")  

if __name__ == "__main__":
    main()


pitchNegative
3D_real_head_dataset/images/ground_truth/pitchNegative_id212


/tmp/ipykernel_22057/3936677592.py:13: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_score = ssim(ground_truth_image,generated, data_range=ground_truth_image.max() -


pitchPositive
yawPositive
3D_real_head_dataset/images/ground_truth/yawPositive_id212


/tmp/ipykernel_22057/3936677592.py:13: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_score = ssim(ground_truth_image,generated, data_range=ground_truth_image.max() -


pitchYawNegative
yawNegative
pitchYawPositive
3D_real_head_dataset/images/ground_truth/pitchYawPositive_id212


/tmp/ipykernel_22057/3936677592.py:13: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_score = ssim(ground_truth_image,generated, data_range=ground_truth_image.max() -


    image_name  pitchNegative  yawPositive  pitchYawPositive
0   00000.jpeg       0.917998     0.917998          0.917998
1   00001.jpeg       0.893537     0.894172          0.873089
2   00002.jpeg       0.889004     0.892519          0.872150
3   00003.jpeg       0.884587     0.889876          0.869198
4   00004.jpeg       0.884138     0.887082          0.863792
..         ...            ...          ...               ...
95  00095.jpeg       0.766096     0.735193          0.641273
96  00096.jpeg       0.764699     0.730839          0.638651
97  00097.jpeg       0.765460     0.730800          0.635814
98  00098.jpeg       0.764779     0.729581          0.636788
99  00099.jpeg       0.763687     0.728584          0.632908

[100 rows x 4 columns]
